In [1]:
from temds import tile
from osgeo import ogr
import geopandas as gpd
import numpy as np
import rioxarray as rio
from temds import worldclim
from temds import crujra
import xarray as xr

from pathlib import Path

from affine import Affine


In [2]:
c_tile = (7, 16)
tile_index = gpd.read_file('/media/rwspicer/data/V3/tem/00-aoi/tile-index/')
hdx = tile_index['H'] == c_tile[0]
vdx = tile_index['V'] == c_tile[1]
bounds = tile_index[vdx & hdx].bounds
minx, maxx, miny, maxy = bounds[['minx','maxx','miny','maxy']].iloc[0]
minx, maxx, miny, maxy

(-1678000.0, -1402000.0, 2915000.0, 3315000.0)

In [3]:
wc_arctic = worldclim.WorldClim('/media/rwspicer/data/V3/tem/02-arctic/worldclim/worldclim-arctic.nc')

In [4]:
wc_arctic.dataset.rio.crs

CRS.from_wkt('PROJCS["WGS 84 / NSIDC EASE-Grid 2.0 North",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",90],PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",SOUTH],AXIS["Northing",SOUTH],AUTHORITY["EPSG","6931"]]')

In [5]:

annual_list = []
for cru_file in list(Path('/media/rwspicer/data/V3/tem/02-arctic/cru-jra/').glob('*.nc')):
    year = int(cru_file.name.split('.')[-4])
    if year < 1970 or year > 1971:
        continue

    temp = crujra.AnnualDaily(year,cru_file, verbose=False)
    # temp.reproject(tile_index.crs.to_wkt())
    annual_list.append(temp)

annual_list[:5]

[CRUJRAnnualDaily: 1971, CRUJRAnnualDaily: 1970]

In [6]:
cru_arctic_ts = crujra.AnnualTimeSeries(annual_list)

In [7]:
cru_arctic_ts[1970:1975]

AnnualTimeSeries
-CRUJRAnnualDaily: 1970
-CRUJRAnnualDaily: 1971

In [8]:
mytile = tile.Tile(c_tile, bounds, 4000, tile_index.crs.to_wkt(), buffer_px=20)

In [9]:
mytile.import_normalized('worldclim', wc_arctic)

In [10]:
mytile.import_normalized('cru_AnnualTimeSeries', cru_arctic_ts)

CRUJRAnnualDaily: 1970 clipping
CRUJRAnnualDaily: 1971 clipping


In [11]:
mytile.data['cru_AnnualTimeSeries']

AnnualTimeSeries
-CRUJRAnnualDaily: 1970
-CRUJRAnnualDaily: 1971

In [12]:
mytile.save('/media/rwspicer/data/V3/tem/03-tiles', overwrite=True)

tmin
tmax
tavg
prec
srad
wind
vapr
tmin
tmax
tmp
pre
dswrf
ugrd
vgrd
spfh
pres
tmin
tmax
tmp
pre
dswrf
ugrd
vgrd
spfh
pres


In [ ]:
mytile.data['cru_AnnualTimeSeries'][1970].dataset.to_netcdf('test.nc', engine='netcdf4')